# Notebook 기본 세팅

In [1]:
# Constant 선언

# 프로젝트 루트 디렉토리를 식별하기 위한 마커 파일 이름
ROOT_MARKER = "pyproject.toml"

# 한글 표시를 위한 나눔바른고딕 폰트 파일 이름
# matplotlib 의 font_manager 에 실제 폰트 파일의 위치를 넣어주어야 한다.
KOREAN_FONT_FILE = "NanumBarunGothic.ttf"

# matplotlib 에서는 font-family 의 이름으로 font 를 설정한다.
# 그래서 font 파일 그 자체가 아니라, 그 파일의 family 이름을 적어준다.
KOREAN_FONT_FAMILY = "NanumBarunGothic"

# 참고
# Font Family 와 Font File 의 차이는,
# Font Family 는 비슷한 디자인 특성을 공유하는 글꼴 그룹을 의미한다.
#
# 예를 들어 '나눔바른고딕' 폰트 패밀리는 일반(Regular), 굵게(Bold), 기울임(Italic) 등 여러 스타일을 포함할 수 있다.
# 반면, 폰트 파일(.ttf, .otf 등)은 이러한 폰트의 하나의 스타일이 저장된 실제 파일이다.
#
# 이 프로젝트에서는 폰트 용량을 줄이기 위해 일반(Regular) 인 NanumBarunGothic.ttf 만 사용한다.

In [2]:
# 프로젝트 root 를 sys.path 에 추가해서 import 구문을 사용하기 쉽게
from pathlib import Path


def find_project_root() -> Path:
    """
    pyproject.toml 파일을 기준으로 루트 디렉토리를 찾는다.
    :return: Path: 프로젝트 루트 디렉토리 경로
    """

    current_path = Path().resolve()

    while current_path != current_path.parent:
        if (current_path / ROOT_MARKER).exists():
            return current_path

        current_path = current_path.parent

    raise FileNotFoundError("프로젝트 루트 디렉토리를 찾을 수 없습니다.")


ROOT_DIR = find_project_root()

In [3]:
# matplotlib 의 한글 font 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt


FONTS_DATA_DIR = ROOT_DIR / "notebooks" / "fonts"


def setup_korean_font():
    font_path = FONTS_DATA_DIR / KOREAN_FONT_FILE
    fm.fontManager.addfont(font_path)

    # 폰트 설정
    plt.rcParams["font.family"] = KOREAN_FONT_FAMILY
    plt.rcParams["axes.unicode_minus"] = False


setup_korean_font()

## Naver Cloud Storage 연동해보기

In [5]:
from dotenv import load_dotenv

load_dotenv()


True

In [6]:
pip install boto3


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

#환경변수 키 가져오기
NCLOUD_ACCESS_KEY = os.getenv("NCLOUD_ACCESS_KEY")
NCLOUD_SECRET_KEY = os.getenv("NCLOUD_SECRET_KEY")
NCLOUD_STORAGE_REGION = os.getenv("NCLOUD_STORAGE_REGION")
NCLOUD_STORAGE_BUCKET = os.getenv("NCLOUD_STORAGE_BUCKET")
NCLOUD_STORAGE_ENDPOINT_URL = os.getenv("NCLOUD_STORAGE_ENDPOINT_URL")

#S3 클라이언트 생성


In [2]:
import boto3
from botocore.client import Config

s3_client = boto3.client(
    "s3",
    endpoint_url=NCLOUD_STORAGE_ENDPOINT_URL,
    aws_access_key_id=NCLOUD_ACCESS_KEY,
    aws_secret_access_key=NCLOUD_SECRET_KEY,
    region_name=NCLOUD_STORAGE_REGION,
    config=Config(signature_version="s3v4"),
)


In [3]:
from io import StringIO

import pandas as pd


DATASETS_DIR = "datasets"


def upload_dataframe_as_csv(df: pd.DataFrame, index: bool = False):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=index)
    return s3_client.put_object(
        Bucket=NCLOUD_STORAGE_BUCKET,
        Key=f"{DATASETS_DIR}/data.csv",
        Body=csv_buffer.getvalue(),
    )

In [4]:
example_df = pd.DataFrame({"name": ["tom", "jane", "elly"], "gender": ["m", "f", "f"], "age": [21, 23, 34]})
example_df

,name,gender,age
0,tom,m,21
1,jane,f,23
2,elly,f,34


In [5]:
upload_dataframe_as_csv(example_df)


{'ResponseMetadata': {'RequestId': '0111b12a-126f-492b-82b5-de15d6abf8b8',
  'HostId': 'Yzk0YmI0YmM5NGJlMjQ4N2RmNTRmYTM2OGNhNTE0NGZjNjMzNmEzYWI4NDZiMGRhMTVjNDFhYjM5NDg3YzcyZg==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'etag': '"84a182d4f332cf1fd04213ecd3c9263e"',
   'x-amz-checksum-crc32': 'jeoaSw==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'x-amz-id-2': 'Yzk0YmI0YmM5NGJlMjQ4N2RmNTRmYTM2OGNhNTE0NGZjNjMzNmEzYWI4NDZiMGRhMTVjNDFhYjM5NDg3YzcyZg==',
   'x-amz-request-id': '0111b12a-126f-492b-82b5-de15d6abf8b8',
   'x-amz-version-id': '922a071a-3c16-11f0-b92f-9cc2c468d112',
   'date': 'Wed, 28 May 2025 22:53:28 GMT',
   'content-length': '0',
   'server': 'Ncloud Storage'},
  'RetryAttempts': 0},
 'ETag': '"84a182d4f332cf1fd04213ecd3c9263e"',
 'ChecksumCRC32': 'jeoaSw==',
 'ChecksumType': 'FULL_OBJECT',
 'VersionId': '922a071a-3c16-11f0-b92f-9cc2c468d112'}

In [6]:
def read_csv_as_dataframe(key: str) -> pd.DataFrame:
    r = s3_client.get_object(Bucket=NCLOUD_STORAGE_BUCKET, Key=key)
    file_content = r["Body"].read().decode("utf-8")
    csv_buffer = StringIO(file_content)
    return pd.read_csv(csv_buffer)

In [7]:
read_csv_as_dataframe(f"{DATASETS_DIR}/data.csv")


,name,gender,age
0,tom,m,21
1,jane,f,23
2,elly,f,34


In [8]:
s3_client.delete_object(Bucket=NCLOUD_STORAGE_BUCKET, Key=f"{DATASETS_DIR}/data.csv")


{'ResponseMetadata': {'RequestId': '5ada3802-e94d-4f7e-85b7-c35cfa9dab73',
  'HostId': 'Yzk0YmI0YmM5NGJlMjQ4N2RmNTRmYTM2OGNhNTE0NGZjNjMzNmEzYWI4NDZiMGRhMTVjNDFhYjM5NDg3YzcyZg==',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-delete-marker': 'true',
   'x-amz-id-2': 'Yzk0YmI0YmM5NGJlMjQ4N2RmNTRmYTM2OGNhNTE0NGZjNjMzNmEzYWI4NDZiMGRhMTVjNDFhYjM5NDg3YzcyZg==',
   'x-amz-request-id': '5ada3802-e94d-4f7e-85b7-c35cfa9dab73',
   'x-amz-version-id': 'a3076700-3c16-11f0-b92f-9cc2c468d112',
   'date': 'Wed, 28 May 2025 22:53:57 GMT',
   'server': 'Ncloud Storage'},
  'RetryAttempts': 0},
 'DeleteMarker': True,
 'VersionId': 'a3076700-3c16-11f0-b92f-9cc2c468d112'}

### -> 추후 container 안에서 연동 코드 작성 및 실행해야함

### 기상청 데이터 수집

In [19]:
import requests
import pandas as pd

#총 96개 지점
stn_ids = [
    "90", "93", "95", "98", "99", "100", "101", "102", "104", "105",
    "106", "108", "112", "114", "115", "119", "121", "127", "129", "130",
    "131", "133", "135", "136", "137", "138", "140", "143", "146", "152",
    "155", "156", "159", "162", "165", "168", "169", "170", "172", "174",
    "177", "184", "185", "188", "189", "192", "201", "202", "203", "211",
    "212", "216", "217", "221", "226", "232", "235", "236", "238", "239",
    "243", "244", "245", "247", "248", "251", "252", "253", "254", "255",
    "257", "258", "259", "260", "261", "262", "263", "264", "266", "268",
    "271", "272", "273", "276", "277", "278", "279", "281", "283", "284",
    "285", "288", "289", "294", "295"
]

years = [
    ("20200101", "20201231"),  # 2020년
    ("20210101", "20211231"),  # 2021년  
    ("20220101", "20221231"),  # 2022년
    ("20230101", "20231231"),  # 2023년
    ("20240101", "20241231")   # 2024년
]

url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"

dfs = []
for start_dt, end_dt in years:
    for stn_id in stn_ids:
        params = {
            "serviceKey": WEATHER_API_KEY,
            "pageNo": "1",
            "numOfRows": "999",
            "dataType": "JSON",
            "dataCd": "ASOS",
            "dateCd": "DAY",
            "startDt": start_dt,
            "endDt": end_dt,
            "stnIds": stn_id
        }
        
        try:
            res = requests.get(url, params=params)
            
            # 응답 상태 먼저 확인
            print(f"지점 {stn_id}, {start_dt[:4]}년 - Status: {res.status_code}")
            
            if res.status_code != 200:
                print(f"❌ HTTP 에러: {res.status_code}")
                continue
                
            # 응답 내용 확인
            if not res.text.strip():
                print(f"❌ 빈 응답")
                continue
                
            response = res.json()
            items = response["response"]["body"]["items"]["item"]
            
        except requests.exceptions.JSONDecodeError as e:
            print(f"🚨 JSON 에러 발생!")
            print(f"   지점: {stn_id}")
            print(f"   기간: {start_dt}-{end_dt}")
            print(f"   응답: {res.text[:200]}")  # 처음 200자만
            print("-" * 50)
            continue  # 다음으로 넘어가기

지점 90, 2020년 - Status: 200
지점 93, 2020년 - Status: 200
지점 95, 2020년 - Status: 200
지점 98, 2020년 - Status: 200
지점 99, 2020년 - Status: 200
지점 100, 2020년 - Status: 200
지점 101, 2020년 - Status: 200
지점 102, 2020년 - Status: 200
지점 104, 2020년 - Status: 200
지점 105, 2020년 - Status: 200
지점 106, 2020년 - Status: 200
지점 108, 2020년 - Status: 200
🚨 JSON 에러 발생!
   지점: 108
   기간: 20200101-20201231
   응답: <OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<returnAuthMsg>HTTP ROUTING ERROR</returnAuthMsg>
		<returnReasonCode>04</returnReasonCode>
	</cmmMsgHeader>
</OpenAPI_Serv
--------------------------------------------------
지점 112, 2020년 - Status: 200
지점 114, 2020년 - Status: 200
지점 115, 2020년 - Status: 200
지점 119, 2020년 - Status: 200
지점 121, 2020년 - Status: 200
지점 127, 2020년 - Status: 200
지점 129, 2020년 - Status: 200
지점 130, 2020년 - Status: 200
지점 131, 2020년 - Status: 200
지점 133, 2020년 - Status: 200
지점 135, 2020년 - Status: 200
지점 136, 2020년 - Status: 200
지점 137, 2020년 - Status: 

In [18]:
import requests
import pandas as pd

#총 96개 지점
stn_ids = [
    "90", "93", "95", "98", "99", "100", "101", "102", "104", "105",
    "106", "108", "112", "114", "115", "119", "121", "127", "129", "130",
    "131", "133", "135", "136", "137", "138", "140", "143", "146", "152",
    "155", "156", "159", "162", "165", "168", "169", "170", "172", "174",
    "177", "184", "185", "188", "189", "192", "201", "202", "203", "211",
    "212", "216", "217", "221", "226", "232", "235", "236", "238", "239",
    "243", "244", "245", "247", "248", "251", "252", "253", "254", "255",
    "257", "258", "259", "260", "261", "262", "263", "264", "266", "268",
    "271", "272", "273", "276", "277", "278", "279", "281", "283", "284",
    "285", "288", "289", "294", "295"
]

years = [
    ("20200101", "20201231"),  # 2020년
    ("20210101", "20211231"),  # 2021년  
    ("20220101", "20221231"),  # 2022년
    ("20230101", "20231231"),  # 2023년
    ("20240101", "20241231")   # 2024년
]

url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"

dfs = []
for start_dt, end_dt in years:
    for stn_id in stn_ids:
        params = {
            "serviceKey": WEATHER_API_KEY,
            "pageNo": "1",
            "numOfRows": "999",
            "dataType": "JSON",
            "dataCd": "ASOS",
            "dateCd": "DAY",
            "startDt": start_dt,
            "endDt": end_dt,
            "stnIds": stn_id
        }

        res = requests.get(url, params=params)
        response = res.json()
        items = response["response"]["body"]["items"]["item"]
        dfs.append(pd.DataFrame(items))
        print(f"{start_dt}-{end_dt} 지점{stn_id}: {len(items)}건")

df = pd.concat(dfs, ignore_index=True)
df.to_csv("weather_96stations_5years.csv", index=False)
print(f"수집 완료: {len(df)}건")

20200101-20201231 지점90: 366건
20200101-20201231 지점93: 366건


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [39]:
response = res.json()
response

{'response': {'header': {'resultCode': '99',
   'resultMsg': '데이터요청은 한번에 최대 1,000건을 넘을 수 없습니다.'}}}

In [28]:
item = response["response"]["body"]["items"]["item"][1]
print(item)
print(type(item))


{'stnId': '108', 'stnNm': '서울', 'tm': '2024-01-02', 'avgTa': '2.9', 'minTa': '2.2', 'minTaHrmt': '0622', 'maxTa': '4.3', 'maxTaHrmt': '1409', 'mi10MaxRn': '', 'mi10MaxRnHrmt': '', 'hr1MaxRn': '', 'hr1MaxRnHrmt': '', 'sumRnDur': '1.58', 'sumRn': '0.0', 'maxInsWs': '5.3', 'maxInsWsWd': '250', 'maxInsWsHrmt': '1407', 'maxWs': '3.1', 'maxWsWd': '270', 'maxWsHrmt': '1409', 'avgWs': '1.6', 'hr24SumRws': '1360', 'maxWd': '290', 'avgTd': '-1.6', 'minRhm': '59', 'minRhmHrmt': '1552', 'avgRhm': '72.6', 'avgPv': '5.5', 'avgPa': '1011.4', 'maxPs': '1024.9', 'maxPsHrmt': '0001', 'minPs': '1020.3', 'minPsHrmt': '2354', 'avgPs': '1022.1', 'ssDur': '9.6', 'sumSsHr': '0.3', 'hr1MaxIcsrHrmt': '1200', 'hr1MaxIcsr': '0.63', 'sumGsr': '3.87', 'ddMefs': '', 'ddMefsHrmt': '', 'ddMes': '', 'ddMesHrmt': '', 'sumDpthFhsc': '', 'avgTca': '8.9', 'avgLmac': '7.4', 'avgTs': '0.3', 'minTg': '-3.3', 'avgCm5Te': '0.0', 'avgCm10Te': '-0.2', 'avgCm20Te': '0.1', 'avgCm30Te': '1.0', 'avgM05Te': '2.8', 'avgM10Te': '6.1', '

In [31]:
items = response["response"]["body"]["items"]["item"]
print(items)

[{'stnId': '108', 'stnNm': '서울', 'tm': '2024-01-01', 'avgTa': '3.3', 'minTa': '-0.3', 'minTaHrmt': '0402', 'maxTa': '7.3', 'maxTaHrmt': '1255', 'mi10MaxRn': '', 'mi10MaxRnHrmt': '', 'hr1MaxRn': '', 'hr1MaxRnHrmt': '', 'sumRnDur': '2.92', 'sumRn': '0.0', 'maxInsWs': '4.5', 'maxInsWsWd': '180', 'maxInsWsHrmt': '1518', 'maxWs': '3.2', 'maxWsWd': '180', 'maxWsHrmt': '1525', 'avgWs': '1.7', 'hr24SumRws': '1435', 'maxWd': '70', 'avgTd': '0.5', 'minRhm': '65', 'minRhmHrmt': '1312', 'avgRhm': '83.4', 'avgPv': '6.3', 'avgPa': '1016.1', 'maxPs': '1029.4', 'maxPsHrmt': '0934', 'minPs': '1024.8', 'minPsHrmt': '2351', 'avgPs': '1026.9', 'ssDur': '9.6', 'sumSsHr': '4.3', 'hr1MaxIcsrHrmt': '1200', 'hr1MaxIcsr': '1.87', 'sumGsr': '7.44', 'ddMefs': '', 'ddMefsHrmt': '', 'ddMes': '2.4', 'ddMesHrmt': '0001', 'sumDpthFhsc': '', 'avgTca': '6.4', 'avgLmac': '5.8', 'avgTs': '-0.2', 'minTg': '-4.7', 'avgCm5Te': '-0.1', 'avgCm10Te': '-0.2', 'avgCm20Te': '0.1', 'avgCm30Te': '1.0', 'avgM05Te': '2.8', 'avgM10Te':

In [32]:
print(type(items))
print(len(items))

<class 'list'>
365


In [33]:
import pandas as pd
df = pd.DataFrame(items)

df.to_csv("asos_daily_2024", index=False)

In [20]:
import os
from dotenv import load_dotenv

load_dotenv()

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

In [21]:
import requests
import pandas as pd

#총 96개 지점
stn_ids = [
    "90", "93", "95", "98", "99", "100", "101", "102", "104", "105",
    "106", "108", "112", "114", "115", "119", "121", "127", "129", "130",
    "131", "133", "135", "136", "137", "138", "140", "143", "146", "152",
    "155", "156", "159", "162", "165", "168", "169", "170", "172", "174",
    "177", "184", "185", "188", "189", "192", "201", "202", "203", "211",
    "212", "216", "217", "221", "226", "232", "235", "236", "238", "239",
    "243", "244", "245", "247", "248", "251", "252", "253", "254", "255",
    "257", "258", "259", "260", "261", "262", "263", "264", "266", "268",
    "271", "272", "273", "276", "277", "278", "279", "281", "283", "284",
    "285", "288", "289", "294", "295"
]

years = [
    ("20200101", "20201231"),  # 2020년
    ("20210101", "20211231"),  # 2021년  
    ("20220101", "20221231"),  # 2022년
    ("20230101", "20231231"),  # 2023년
    ("20240101", "20241231")   # 2024년
]

url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"

dfs = []
for start_dt, end_dt in years:
    for stn_id in stn_ids:
        params = {
            "serviceKey": WEATHER_API_KEY,
            "pageNo": "1",
            "numOfRows": "999",
            "dataType": "JSON",
            "dataCd": "ASOS",
            "dateCd": "DAY",
            "startDt": start_dt,
            "endDt": end_dt,
            "stnIds": stn_id
        }
        
        try:
            res = requests.get(url, params=params)
            response = res.json()
            
            # 안전한 데이터 추출
            if (response.get("response", {}).get("header", {}).get("resultCode") == "00" and
                "body" in response.get("response", {}) and 
                "items" in response["response"]["body"]):
                
                items = response["response"]["body"]["items"]["item"]
                
                # items 형태 정규화
                if items is None:
                    items = []
                elif isinstance(items, dict):  # 단일 딕셔너리
                    items = [items]
                elif not isinstance(items, list):  # 기타 형태
                    items = []
                
                # DataFrame 생성 (빈 리스트도 처리)
                if items:
                    df_temp = pd.DataFrame(items)
                    dfs.append(df_temp)
                    print(f"✅ {start_dt}-{end_dt} 지점{stn_id}: {len(items)}건")
                else:
                    print(f"⚠️  {start_dt}-{end_dt} 지점{stn_id}: 데이터 없음")
            else:
                # API 에러 응답
                error_msg = response.get("response", {}).get("header", {}).get("resultMsg", "Unknown")
                print(f"❌ {start_dt}-{end_dt} 지점{stn_id}: {error_msg}")
                
        except requests.exceptions.JSONDecodeError:
            print(f"🚨 JSON 에러 - 지점{stn_id}, {start_dt}")
            continue
        except Exception as e:
            print(f"🚨 기타 에러 - 지점{stn_id}, {start_dt}: {str(e)[:50]}")
            continue

# 최종 처리
if dfs:
    df = pd.concat(dfs, ignore_index=True)
    df.to_csv("weather_96stations_5years.csv", index=False)
    print(f"🎉 수집 완료: {len(df)}건")
else:
    print("❌ 수집된 데이터가 없습니다.")

✅ 20200101-20201231 지점90: 366건
✅ 20200101-20201231 지점93: 366건
✅ 20200101-20201231 지점95: 366건
✅ 20200101-20201231 지점98: 366건
✅ 20200101-20201231 지점99: 366건
✅ 20200101-20201231 지점100: 366건
✅ 20200101-20201231 지점101: 366건
✅ 20200101-20201231 지점102: 366건
✅ 20200101-20201231 지점104: 366건
✅ 20200101-20201231 지점105: 366건
✅ 20200101-20201231 지점106: 366건
✅ 20200101-20201231 지점108: 366건
✅ 20200101-20201231 지점112: 366건
✅ 20200101-20201231 지점114: 366건
✅ 20200101-20201231 지점115: 366건
✅ 20200101-20201231 지점119: 366건
✅ 20200101-20201231 지점121: 366건
✅ 20200101-20201231 지점127: 366건
✅ 20200101-20201231 지점129: 366건
✅ 20200101-20201231 지점130: 366건
✅ 20200101-20201231 지점131: 366건
✅ 20200101-20201231 지점133: 366건
✅ 20200101-20201231 지점135: 366건
✅ 20200101-20201231 지점136: 366건
✅ 20200101-20201231 지점137: 366건
✅ 20200101-20201231 지점138: 366건
✅ 20200101-20201231 지점140: 366건
✅ 20200101-20201231 지점143: 366건
✅ 20200101-20201231 지점146: 366건
✅ 20200101-20201231 지점152: 366건
✅ 20200101-20201231 지점155: 366건
✅ 20200101-20